# Problem Statement:
We need to build a model that predicts the transistor current value based on some parameters. 

Here is the list of parameters that you are allowed to use in your model as model input:
* vds (Continuous Value)
* L(um) (Continuous Value)
* W(um) (Continuous Value)
* drain_length(um) (Continuous Value)
* temperature (Continuous Value)
* vgs (Continuous Value)
* vsb (Continuous Value)
* corner (Categorical Value)

We need to predict the following value:
* id(uA) (Continuous value)

Please drop all other columns in the dataset.

The goal is optimize for minimum mean absolute percentage error for Id(uA).

You must use tensorflow or pytorch to implement this (Deep Learning). 

Try to be innovative on how to choose the model architecture based on the known physical background around MOS transistors.


Your submission will be assessed based on the following criteria in priority order:
1. Code cleanness and readability.
2. Code documentation.
3. General documentation and usage documentation.
4. Model quality and choice decision reasoning.
5. Model performance.

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense, Reshape, Normalization

In [67]:
# explore data
df = pd.read_csv('simulated_data_nmos.csv')
print("table shape = {}\n".format(df.shape))
list(df.columns)


table shape = (871488, 23)



['vds',
 'L(um)',
 'W(um)',
 'drain_length(um)',
 'temperature',
 'vgs',
 'vsb',
 'vth',
 'v_pinch_off',
 'lambda(um)',
 'id(uA)',
 'ig(aA)',
 'gm(uS)',
 'gmb(uS)',
 'ro(Kohm)',
 'cgg(aF)',
 'cgs(aF)',
 'cgd(aF)',
 'cdd(aF)',
 'cdb(aF)',
 'cgb(aF)',
 'csb(aF)',
 'corner']

In [68]:
df["corner"].unique()

array(['tt', 'ss', 'sf', 'fs', 'ff'], dtype=object)

In [69]:
inputs = df[["vds","L(um)","W(um)","drain_length(um)","temperature","vgs","vsb","corner"]]
labels = df["id(uA)"]
inputs["corner"].replace(['tt', 'ss', 'sf', 'fs', 'ff'], [1,2,3,4,5], inplace=True)
print(inputs.head)

<bound method NDFrame.head of          vds  L(um)  W(um)  drain_length(um)  temperature  vgs  vsb  corner
0      -0.45   8.00   0.84               0.3         27.0  0.0  0.0       1
1      -0.40   8.00   0.84               0.3         27.0  0.0  0.0       1
2      -0.35   8.00   0.84               0.3         27.0  0.0  0.0       1
3      -0.30   8.00   0.84               0.3         27.0  0.0  0.0       1
4      -0.25   8.00   0.84               0.3         27.0  0.0  0.0       1
...      ...    ...    ...               ...          ...  ...  ...     ...
871483  2.30   0.15   7.00               0.3         27.0  1.8  1.5       5
871484  2.35   0.15   7.00               0.3         27.0  1.8  1.5       5
871485  2.40   0.15   7.00               0.3         27.0  1.8  1.5       5
871486  2.45   0.15   7.00               0.3         27.0  1.8  1.5       5
871487  2.50   0.15   7.00               0.3         27.0  1.8  1.5       5

[871488 rows x 8 columns]>


C:\Users\nessm\AppData\Local\Temp\ipykernel_18168\4088935857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inputs["corner"].replace(['tt', 'ss', 'sf', 'fs', 'ff'], [1,2,3,4,5], inplace=True)


In [70]:
len(inputs)

871488

In [71]:
frame_size = 128
no_of_frames = int(len(inputs)/frame_size) 
inputs.shape[1]

8

In [72]:
inputs = np.array(inputs)
print(inputs.shape)
inputsFramed = inputs[:no_of_frames*frame_size,:]
inputsFramed = inputsFramed.reshape(no_of_frames,frame_size,inputs.shape[1],1)
print(inputsFramed.shape) # (no. of frames, frame size, 9)

(871488, 8)
(6808, 128, 8, 1)


In [73]:
labels = np.array(labels)
print(labels.shape)
labels = labels[:no_of_frames*frame_size]
labels = labels.reshape(no_of_frames,frame_size,1,1)
print(labels.shape) # (no. of frames, frame size, 1)

(871488,)
(6808, 128, 1, 1)


In [74]:
# Split the data into training and validation/test sets
train_inputs, valtest_inputs, train_labels, valtest_labels = train_test_split(inputsFramed, labels, test_size=0.2, random_state=42)
    
# Split the validation/test set into separate validation and test sets
val_inputs, test_inputs, val_labels, test_labels = train_test_split(valtest_inputs, valtest_labels, test_size=0.5, random_state=42)


In [75]:
train_inputs.shape

(5446, 128, 8, 1)

In [118]:
# Reshape the input shape
input_shape = (128, 8, 1)  # Adjusted input shape

# Create the model
model = tf.keras.models.Sequential()
model.add(Normalization(axis=-1, input_shape=input_shape))  # Normalization layer
model.add(Conv2D(16, (8, 8), activation='relu', padding='same', use_bias=False))
model.add(Dropout(0.2))

# Flatten the output
model.add(Flatten())

# Add a dense layer for regression
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer without activation function for continuous value prediction

# Compile the model
model.compile(optimizer='adam', metrics=['accuracy'], loss="mean_absolute_error")
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5)

# Print the model summary
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_9 (Normaliza  (None, 128, 8, 1)         3         
 tion)                                                           
                                                                 
 conv2d_9 (Conv2D)           (None, 128, 8, 16)        1024      
                                                                 
 dropout_15 (Dropout)        (None, 128, 8, 16)        0         
                                                                 
 flatten_6 (Flatten)         (None, 16384)             0         
                                                                 
 dense_15 (Dense)            (None, 64)                1048640   
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                     

In [119]:
history = model.fit(train_inputs, train_labels, validation_data=(val_inputs, val_labels), verbose=0,validation_split=0.2, epochs=100, batch_size=16, callbacks=[early_stopping])

In [120]:
test_loss, test_acc = model.evaluate(inputsFramed, labels)
print('Test accuracy: ', test_acc)

213/213 [==============================] - 1s 5ms/step - loss: 338.4404 - accuracy: 0.0000e+00
Test accuracy:  0.0


In [121]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
1,343.493103,0.000013,277.056213,0.0,1
2,343.443115,0.000014,276.776215,0.0,2
3,343.408966,0.000015,277.587585,0.0,3
4,343.419250,0.000024,276.854797,0.0,4
5,343.494476,0.000012,277.128662,0.0,5
